In [1]:
import pandas as pd
import sqlite3

In [2]:
! ls ../unzippedData

bom.movie_gross.csv  rt.movie_info.tsv    tmdb.movies.csv
im.db                rt.reviews.tsv       tn.movie_budgets.csv


In [3]:
MovieBudget = pd.read_csv('../unzippedData/tn.movie_budgets.csv')
MovieBudget

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [4]:
def clean_number_series(Series):
    Series = Series.str.replace(",", "")
    Series = Series.str.replace("$", "")
    Series = Series.astype(int)
    return Series

def clean_column_headers(dataframe):
    dataframe.columns = dataframe.columns.str.title()
    dataframe.columns = dataframe.columns.str.replace("_", " ")

In [5]:
clean_column_headers(MovieBudget)
MovieBudget['Worldwide Gross'] = clean_number_series(MovieBudget['Worldwide Gross'])
MovieBudget['Domestic Gross'] = clean_number_series(MovieBudget['Domestic Gross'])
MovieBudget['Production Budget'] = clean_number_series(MovieBudget['Production Budget'])


/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_82010/508723308.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Series = Series.str.replace("$", "")


In [11]:
# Making Profitability Column

MovieBudget['Profitability'] = MovieBudget['Worldwide Gross'] + MovieBudget['Domestic Gross'] - MovieBudget['Production Budget']
MovieBudget['Revenue Budget Ratio'] = (MovieBudget['Worldwide Gross'] + MovieBudget['Domestic Gross']) / MovieBudget['Production Budget']

MovieBudget.head()

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,Revenue Budget Ratio
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,3111852904,8.322007
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,876127750,3.133774
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,-157475300,0.550071
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,1531419831,5.632244
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,1619903129,6.110105


In [40]:
MovieBudgetFiltered = MovieBudget[(MovieBudget['Production Budget'] < 40000000) & (MovieBudget['Production Budget'] > 0)]
MovieBudgetFiltered.sort_values('Revenue Budget Ratio', ascending=False, inplace=True)
MovieBudgetFiltered.reset_index(drop=True, inplace=True)
MovieBudgetFiltered[:50]
MovieBudgetFiltered['Release Date'] = MovieBudgetFiltered['Release Date'].str[-4:].astype(int)
MovieBudgetFiltered= MovieBudgetFiltered[(MovieBudgetFiltered['Release Date']>2005)].reset_index(drop=True)
MovieBudgetFiltered[:50]

/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_82010/3106175832.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MovieBudgetFiltered.sort_values('Revenue Budget Ratio', ascending=False, inplace=True)
/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_82010/3106175832.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MovieBudgetFiltered['Release Date'] = MovieBudgetFiltered['Release Date'].str[-4:].astype(int)


,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,Revenue Budget Ratio
0,93,2009,Paranormal Activity,450000,107918810,194183034,301651844,671.337431
1,80,2015,The Gallows,100000,22764410,41656474,64320884,644.208840
2,57,2007,Once,150000,9445857,23323631,32619488,218.463253
3,81,2006,Facing the Giants,100000,10178331,10243159,20321490,204.214900
4,12,2012,The Devil Inside,1000000,53262945,101759490,154022435,155.022435
5,29,2008,Fireproof,500000,33456317,33473297,66429614,133.859228
6,90,2014,Godâs Not Dead,1150000,60755732,63777092,123382824,108.289412
7,63,2011,Insidious,1500000,54009150,99870886,152380036,102.586691
8,14,2015,Unfriended,1000000,32789645,64364198,96153843,97.153843
9,60,2009,Home,500000,15433,44793168,44308601,89.617202


In [ ]:
# join this table with directors, genre and actors